In [4]:
import tensorflow as tf

tf.set_random_seed(777)  # for reproducibility

# x_data 와 y_data의 개념은??
# 학습시키고자하는 훈련데이터와 결과 데이터.

x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

# None개 있는대로 주세요? 4개로 구성된 tensor들이 None개 들어있다.
# placeholder가 왜 사용되었나?
X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
# 몃 개의 레이블로 설정되어 있는가?
nb_classes = 3

# 정규분포(normal distribution) - 통계 확률분포에서 가장 중요한 분포로 종 모양(bell shpae)으로 표현되는 확률
# 균등(균일)분포(uniform distribution) - 각각의 구간에서 동일한 확률로 표현되는 분포로 사각형 모양
# 잘린(절단)정규분포(truncated normal distribution) - 정규분포에서 일부 구간을 잘라낸 분포

# 가중치와 편향값을 설정하는데, 입력과 출력의 shape를 설정해서 초기값을 설정,이름을 지정(Optional)
# random_normal는 정규분포로부터 랜덤 값을 아웃한다.
# truncated_normal
# tf.random_uniform
W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')


# What is Softmax ?
# 활성화 함수. 회귀가 아닌 분류에서 활용되는 함수,
# 출력층에서 활성화시키기 위해서 사용되는 함수. 출력의 확률로 해석할 수 있게 만든다.
# 확률로 판단해서 맞고 틀림을 구별한다.
# 여러개의 클래스를 활용할 때 softmax를 활용한다.

# http://www.datamarket.kr/xe/board_LCmL04/26153
#   H(x) = Wx + b
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
# 만약 softmax가 없다면 단일 tensor만 확인가능한 hypo로 나올 것이다.
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
# 손실함수로 활용되는 부분. 즉 답과 결과가 얼마나 떨어져있는가? 를 파악할 수 있는 부분.
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))

# 얼마나 떨어져있는가를 보고 조금씩 움직이면서 값을 교정한다?
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)


# 출력을 주의깊게 봐야함.
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(2001):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    print('--------------')

#   Testing & One-hot encoding
#    [[  2.94248015e-03   9.97048199e-01   9.42312636e-06]] [1]
# tf.argmax(a, 1) 첫번째 인자는 축을 말하고 1은 하나를 보여달라!
    a = sess.run(hypothesis, feed_dict={X: [[1, 11, 7, 9]]})
    print(a, sess.run(tf.argmax(a, 1)))

    print('--------------')

    b = sess.run(hypothesis, feed_dict={X: [[1, 3, 4, 3]]})
    print(b, sess.run(tf.argmax(b, 1)))

    print('--------------')

    c = sess.run(hypothesis, feed_dict={X: [[1, 1, 0, 1]]})
    print(c, sess.run(tf.argmax(c, 1)))

    print('--------------')

    # 복수개의 feed_dict도 가능하다.
    all = sess.run(hypothesis, feed_dict={
                   X: [[1, 11, 7, 9], [1, 3, 4, 3], [1, 1, 0, 1]]})
    print(all, sess.run(tf.argmax(all, 1)))


0 2.77622
200 0.510587
400 0.417496
600 0.342975
800 0.269481
1000 0.227048
1200 0.206008
1400 0.188453
1600 0.17358
1800 0.16082
2000 0.149758
--------------
[[  1.30267430e-03   9.98687923e-01   9.43692248e-06]] [1]
--------------
[[ 0.93165535  0.06082547  0.00751921]] [0]
--------------
[[  1.16895160e-08   3.14100587e-04   9.99685884e-01]] [2]
--------------
[[  1.30267430e-03   9.98687923e-01   9.43692248e-06]
 [  9.31655347e-01   6.08254708e-02   7.51920976e-03]
 [  1.16895160e-08   3.14100587e-04   9.99685884e-01]] [1 0 2]
